In [1]:
import tensorflow as tf
import numpy as np
import DataGenerator

In [2]:
learning_rate = 0.0001
batch_size = 2
prefetch = 2
num_epochs = 10
num_gpus = 2

In [36]:
def input_fn():
    # trainTransforms = [
    #     DataGenerator.Normalization(),
    #     DataGenerator.RandomCrop(),
    #     DataGenerator.RandomNoise(prob),
    #     DataGenerator.BSplineTransform(2, 15, prob),
    #     DataGenerator.RandomTranslation(prob)
    #     ]

    trainTransforms = [
        DataGenerator.Normalization(),
        DataGenerator.BSplineTransform(2, 15, prob)
        ]

    TrainDataset = DataGenerator.DataGenerator(
        data_dir='train-data',
        transforms=trainTransforms,
        train=True
        )

    trainDataset = TrainDataset.get_dataset()
    trainDataset = trainDataset.shuffle(buffer_size=5)
    trainDataset = trainDataset.batch(batch_size)
    trainDataset = trainDataset.prefetch(prefetch)

    iterator = tf.data.Iterator.from_structure(trainDataset.output_types, trainDataset.output_shapes)
    features, labels = iterator.get_next()
#     training_init_op = iterator.make_initializer(trainDataset)

    return features, labels

In [37]:
def conv3d(x, no_of_input_channels, no_of_filters, filter_size, strides, padding, name):
    with tf.variable_scope(name) as scope:
        filter_size.extend([no_of_input_channels, no_of_filters])
        weights = tf.Variable(tf.truncated_normal(filter_size, stddev=0.05), name='weights')
        biases = tf.Variable(tf.truncated_normal([no_of_filters]), name='biases')
        conv = tf.nn.conv3d(x, weights, strides=strides, padding=padding, name=name)
        conv += biases
        
        return conv

In [38]:
def upsamp(x, no_of_kernels, name):
    with tf.variable_scope(name) as scope:
        return tf.layers.conv3d_transpose(x, no_of_kernels, [2,2,1], 2, padding='VALID', use_bias=True, reuse=tf.AUTO_REUSE)
    
    

In [39]:
def prelu(x, scope=None):
    with tf.variable_scope(name_or_scope=scope, default_name="prelu", reuse=tf.AUTO_REUSE):
        alpha = tf.get_variable("prelu", shape=x.get_shape()[-1], dtype=_x.dtype, initializer=tf.constant_initializer(0.1))
        return tf.maximum(0.0, x) + alpha * tf.minimum(0.0, x)
    
    

In [40]:
def graph_encoder(x):
    
    fine_grained_features = {}
    
    conv1 = conv3d(x,1,16,[5,5,5],[1,1,1,1,1],'SAME','Conv1')
    conv1 = prelu(conv1,'prelu1')
    
    res1 = tf.add(x,conv1)
    fine_grained_features['res1'] = res1
    
    down1 = conv3d(res1,16,32,[2,2,1],[1,2,2,2,1],'VALID','DownSampling1')
    down1 = prelu(down1,'down_prelu1')
    
    conv2 = conv3d(down1,32,32,[5,5,5],[1,1,1,1,1],'SAME','Conv2')
    conv2 = prelu(conv2,'prelu2')
    
    conv3 = conv3d(conv2,32,32,[5,5,5],[1,1,1,1,1],'SAME','Conv3')
    conv3 = prelu(conv3,'prelu3')
    
    res2 = tf.add(down1,conv3)
    fine_grained_features['res2'] = res2

    down2 = conv3d(res2,32,64,[2,2,1],[1,2,2,2,1],'VALID','DownSampling2')
    down2 = prelu(down2,'down_prelu2')
    
    conv4 = conv3d(down2,64,64,[5,5,5],[1,1,1,1,1],'SAME','Conv4')
    conv4 = prelu(conv4,'prelu4')
    
    conv5 = conv3d(conv4,64,64,[5,5,5],[1,1,1,1,1],'SAME','Conv5')
    conv5 = prelu(conv5,'prelu5')
    
    conv6 = conv3d(conv5,64,64,[5,5,5],[1,1,1,1,1],'SAME','Conv6')
    conv6 = prelu(conv6,'prelu6')
    
    res3 = tf.add(down2,conv6)
    fine_grained_features['res3'] = res3

    down3 = conv3d(res3,64,128,[2,2,1],[1,2,2,2,1],'VALID','DownSampling3')
    down3 = prelu(down3,'down_prelu3')
    
    conv7 = conv3d(down3,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv7')
    conv7 = prelu(conv7,'prelu7')
    
    conv8 = conv3d(conv7,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv8')
    conv8 = prelu(conv8,'prelu8')
    
    conv9 = conv3d(conv8,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv9')
    conv9 = prelu(conv9,'prelu9')
    
    res4 = tf.add(down3,conv9)
    fine_grained_features['res4'] = res4

    down4 = conv3d(res4,128,256,[2,2,1],[1,2,2,2,1],'VALID','DownSampling4')
    down4 = prelu(down4,'down_prelu4')
    
    conv10 = conv3d(down4,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv10')
    conv10 = prelu(conv10,'prelu10')
    
    conv11 = conv3d(conv10,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv11')
    conv11 = prelu(conv11,'prelu11')
    
    conv12 = conv3d(conv11,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv12')
    conv12 = prelu(conv12,'prelu12')
    
    res5 = tf.add(down4,conv12)
    fine_grained_features['res5'] = res5
    
    return fine_grained_features

In [41]:
def graph_decoder(features, labels):
    
    inp = features['res5']
    
    upsamp1 = upsamp(inp,128,'Upsampling1')
    upsamp1 = prelu(upsamp1,'prelu_upsamp1')
    
    concat1 = tf.concat([upsamp1,features['res4']],axis=4)
    
    conv13 = conv3d(concat1,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv13')
    conv13 = prelu(conv13,'prelu13')
    
    conv14 = conv3d(conv13,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv14')
    conv14 = prelu(conv14,'prelu14')
    
    conv15 = conv3d(conv14,256,256,[5,5,5],[1,1,1,1,1],'SAME','Conv15')
    conv15 = prelu(conv15,'prelu15')
    
    res6 = tf.add(concat1,conv15)
    
    upsamp2 = upsamp(res6,64,'Upsampling2')
    upsamp2 = prelu(upsamp2,'prelu_upsamp2')
    
    concat2 = tf.concat([upsamp2,features['res3']],axis=4)
    
    conv16 = conv3d(concat2,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv16')
    conv16 = prelu(conv16,'prelu16')
    
    conv17 = conv3d(conv16,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv17')
    conv17 = prelu(conv17,'prelu17')
    
    conv18 = conv3d(conv17,128,128,[5,5,5],[1,1,1,1,1],'SAME','Conv18')
    conv18 = prelu(conv18,'prelu18')
    
    res7 = tf.add(concat2,conv18)
    
    upsamp3 = upsamp(res7,32,'Upsampling3')
    upsamp3 = prelu(upsamp3,'prelu_upsamp3')
    
    concat3 = tf.concat([upsamp3,features['res2']],axis=4)
    
    conv19 = conv3d(concat3,64,64,[5,5,5],[1,1,1,1,1],'SAME','Conv19')
    conv19 = prelu(conv19,'prelu19')
    
    conv20 = conv3d(conv19,64,64,[5,5,5],[1,1,1,1,1],'SAME','Conv20')
    conv20 = prelu(conv20,'prelu20')
    
    res8 = tf.add(concat3,conv20)
    
    upsamp4 = upsamp(res8,16,'Upsampling4')
    upsamp4 = prelu(upsamp4,'prelu_upsamp4')
    
    concat4 = tf.concat([upsamp4,features['res1']],axis=4)
    
    conv21 = conv3d(concat4,32,32,[5,5,5],[1,1,1,1,1],'SAME','Conv21')
    conv21 = prelu(conv21,'prelu21')
    
    res9 = tf.add(concat4,conv21)
    
    conv22 = conv3d(res9,32,2,[1,1,1],[1,1,1,1,1],'SAME','Conv22')
    conv22 = prelu(conv22,'prelu22')
    
    return conv22

In [42]:
def model_fn(features, labels, mode):
    encoded = graph_encoder(features)
    decoded = graph_decoder(encoded, labels)

    predictions = {
          "classes": tf.argmax(input=tf.reshape(decoded, [-1, 2]), axis=1),
          "probabilities": tf.nn.softmax(tf.reshape(decoded, [-1, 2]), axis=1, name="softmax_tensor")
       }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    with tf.variable_scope('cost') as scope:
        cost = tf.divide(tf.multiply(2.0,tf.reduce_sum(tf.multiply(decoded, labels))),
                         tf.add(tf.reduce_sum(tf.square(decoded)), tf.reduce_sum(tf.square(labels))))    

    if mode == tf.estimator.ModeKeys.TRAIN:
        with tf.variable_scope('optimizer') as scope:
            opt = tf.train.AdamOptimizer(learning_rate).maximize(cost, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=cost, train_op=opt)

    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}

    return tf.estimator.EstimatorSpec(mode=mode, loss=cost, eval_metric_ops=eval_metric_ops)

In [43]:
run_config = tf.estimator.RunConfig(
  save_checkpoints_steps=2,
  tf_random_seed=0,
  model_dir='tmp/'
)

estimator = tf.estimator.Estimator(model_fn=model_fn, config=run_config)
estimator.train(input_fn, steps=num_steps)

INFO:tensorflow:Using config: {'_model_dir': 'tmp/', '_tf_random_seed': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': 2, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1827b04f60>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
a = np.random.randint(0,10,(128,128,64)).astype(np.float32).reshape((1,128,128,64,1))
c = np.random.randint(0,10,(128,128,64)).astype(np.float32).reshape((1,128,128,64,1))

In [ ]:
b = None
encoded = graph_encoder(a)
decoded = graph_decoder(encoded,c)
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)

    sess.run( tf.global_variables_initializer())
    b = sess.run(decoded['gradients'])
    writer.close()

In [34]:
b

NameError: name 'b' is not defined

In [ ]:
tf.estimator.TrainSpec()